In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import sympy as sym

In [ ]:
def GetNewton(f,df,xn,itmax=1000,precision=1e-12):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [ ]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x: 
        root = GetNewton(f,df,i)
        if root != False:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [ ]:
x=sym.Symbol("x",real=True)
y=sym.Symbol("x",real=True)

In [ ]:
def GetHermite(x,y,n):
    
    y=sym.exp(-(x**2))
    
    Pol=sym.exp(x**2)*sym.diff(y,x,n)
    Pol=(-1)**n * Pol
    
    return sym.simplify(Pol)

In [ ]:
Hermite=[]
DevHermite=[]
deg=20

In [ ]:
for i in range(0,deg+1):
    
    Pol=GetHermite(x,y,i)
    DevPol=sym.diff(Pol,x,1)
    Hermite.append(Pol)
    DevHermite.append(DevPol)

In [ ]:
def GetAllRoots(grado,xn,Hermite,DevHermite):
    
    pol = sym.lambdify([x],Hermite[grado],'numpy')
    Devpol = sym.lambdify([x],DevHermite[grado],'numpy')
    Roots = GetRoots(pol,Devpol,xn)
    
    return Roots

In [ ]:
def Get_Weights(xi,n):
    
    Hn= sym.lambdify([x],Hermite[n-1],'numpy')
    
    num=2**(n-1)*np.math.factorial(n)*np.sqrt(np.pi)
    den=n**2*Hn(xi)**2
    
    return num/den

In [ ]:
xn=np.linspace(-10,10,100)
NumRoots=(GetAllRoots(deg,xn,Hermite,DevHermite))
NumWeights=Get_Weights(NumRoots,deg)
Np_Roots,Np_Weights = np.polynomial.hermite.hermgauss(deg)

In [ ]:
for i in range(len(NumRoots)):
    print(NumRoots[i],NumWeights[i])

In [ ]:
plt.scatter(NumRoots,NumWeights)
plt.xlabel("Raices")
plt.ylabel("Pesos")
plt.show()

In [ ]:
def integrate_Hermite(f,a,b,n,Rootsi,Weightsi):
    t=Rootsi
    integral=np.sum(Weightsi*f(t,n)*np.exp(t**2))
    
    return integral

In [ ]:
def function(xn,n):
    Hn= sym.lambdify([x],Hermite[n],'numpy')    
    psi=(np.pi**(-1/4)*np.exp(-xn**2/2)*Hn(xn))/(np.sqrt(2**n * np.math.factorial(n)))

    return np.abs(psi)**2 *xn**2

In [ ]:
a=-100000
b=100000

Integral=integrate_Hermite(function,a,b,1,NumRoots,NumWeights)

In [ ]:
print("Al usar Gauss-Hermite la aproximación de la integral es",Integral)